In [1]:
import tensorflow as tf
from tensorflow import keras

### Use @tf.function
In tensorflow you can use tf.function annotation to convert a common python method to a tensorflow graph <br>
It is also important to use the features of tf.function to get info f model and inference the trained models

In [2]:
@tf.function
def cube(z):
    return tf.pow(z, 3)

**Pass int and float type to this funcion, it will return different tensor data type, in production it could be a huge problem!**

<br>
tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)
<br>
tf.Tensor([ 1.  8. 27.], shape=(3,), dtype=float32)

In [5]:
print(cube(tf.constant([1, 2, 3])))
print(cube(tf.constant([1., 2., 3.])))

tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)
tf.Tensor([ 1.  8. 27.], shape=(3,), dtype=float32)


### Constrain function parameter with explizit type

In [7]:
@tf.function(input_signature=[tf.TensorSpec(shape=[None], 
                                            dtype = tf.int32, 
                                            name = 'x')])
def cube(z):
    return tf.pow(z, 3)

In [8]:
# float is not allowed anymore 
try:
    print(cube(tf.constant([1., 2., 3.])))
except ValueError as ex:
    print(ex)

Python inputs incompatible with input_signature: inputs ((<tf.Tensor: id=23, shape=(3,), dtype=float32, numpy=array([1., 2., 3.], dtype=float32)>,)), input_signature ((TensorSpec(shape=(None,), dtype=tf.int32, name='x'),))


In [9]:
try:
    print(cube(tf.constant([1, 2, 3])))
except ValueError as ex:
    print(ex)

tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)


### Functions which annotated with @tf.function can be exported as SavedModel by using get_concrete_function

In [13]:
# tf.function -> python function -> tensorflow graph
# get_concreate_function -> add input signature -SavedModel

cube_func_int32 = cube.get_concrete_function(tf.TensorSpec(shape=[None], 
                                            dtype = tf.int32, 
                                            name = 'x'))

cube_func_int32

In [14]:
cube_func_int32.graph

In [16]:
cube_func_int32.graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'Pow/y' type=Const>,
 <tf.Operation 'Pow' type=Pow>,
 <tf.Operation 'Identity' type=Identity>]

In [26]:
# <tf.Operation 'Pow' type=Pow> is the operation that we implmented above
# we can look on it in detail
pow_op = cube_func_int32.graph.get_operations()[2] # the second from list

print(pow_op)

name: "Pow"
op: "Pow"
input: "x"
input: "Pow/y"
attr {
  key: "T"
  value {
    type: DT_INT32
  }
}



In [25]:
print(list(pow_op.inputs))
print(list(pow_op.outputs))
# in results, we can see x:0, Pow:0 are the first two operations 
# of cube_func_int32.graph.get_operations()

[<tf.Tensor 'x:0' shape=(None,) dtype=int32>, <tf.Tensor 'Pow/y:0' shape=() dtype=int32>]
[<tf.Tensor 'Pow:0' shape=(None,) dtype=int32>]


### Print all infos about this graph

In [32]:
cube_func_int32.graph.as_graph_def()

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
node {
  name: "Pow/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 3
      }
    }
  }
}
node {
  name: "Pow"
  op: "Pow"
  input: "x"
  input: "Pow/y"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "Identity"
  op: "Identity"
  input: "Pow"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
versions {
  producer: 55
}

In [28]:
cube_func_int32.graph.get_operation_by_name('x')

<tf.Operation 'x' type=Placeholder>

In [30]:
cube_func_int32.graph.get_tensor_by_name('x:0')

<tf.Tensor 'x:0' shape=(None,) dtype=int32>